In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [3]:
#q1
q1 = bq.query(
"""
select geo_id, county_name
from bigquery-public-data.geo_us_boundaries.counties
where county_name = 'Dane'
""", job_config=no_cache)
df = q1.to_dataframe()["geo_id"][0]
df

'55025'

In [4]:
#q2
q2 = bq.query(
"""
SELECT state_fips_code, COUNT(*) as county_count
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY county_count DESC
LIMIT 5""", job_config=no_cache)
df = q2.to_dataframe()
dict = df.set_index('state_fips_code')['county_count'].to_dict()
dict

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [5]:
#q3
total_bytes_billed_q1 = q1.total_bytes_billed
total_bytes_billed_q2 = q2.total_bytes_billed

cost_per_tb = 6.25 
bytes_per_tb = 2 ** 40


cost_per_byte = cost_per_tb / bytes_per_tb

if total_bytes_billed_q1:
    cost_q1 = total_bytes_billed_q1 * cost_per_byte
else:
    cost_q1 = 0

if total_bytes_billed_q2:
    cost_q2 = total_bytes_billed_q2 * cost_per_byte
else:
    cost_q2 = 0

cost_dict = {'q1':cost_q1, 'q2': cost_q2}
cost_dict

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

In [6]:
#q4
ds = bigquery.Dataset("cs-544-398518.p8")
bq.delete_dataset(ds, delete_contents = True, not_found_ok = True)
bq.create_dataset(ds, exists_ok=True)
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
url = "gs://cs544_p8vishmeet1/hdma-wi-2021.parquet"
job = bq.load_table_from_uri(url, "cs-544-398518.p8.hdma", job_config=config)
[ds.dataset_id for ds in bq.list_datasets("cs-544-398518")]

['p8']

In [7]:
import time
time.sleep(10)

In [8]:
#q5
q = bq.query("""
SELECT c.county_name, COUNT(*) as loan_applications_count
FROM cs-544-398518.p8.hdma as h
INNER JOIN bigquery-public-data.geo_us_boundaries.counties as c ON h.county_code = c.county_fips_code
GROUP BY c.county_name
ORDER BY loan_applications_count DESC
LIMIT 10
""")
df = q.to_dataframe()

result_dict = df.set_index('county_name')['loan_applications_count'].to_dict()
result_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [9]:
#q6
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(ds.table("applications"))
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

q = bq.query("""
SELECT COUNT(*) as COUNT 
FROM cs-544-398518.p8.applications
WHERE income = 110000
""")
q.to_dataframe()["COUNT"][0]

1

In [10]:
#q7
q = bq.query("""
SELECT c.county_name, COUNT(*) as count
FROM `cs-544-398518.p8.applications` as a
JOIN `bigquery-public-data.geo_us_boundaries.counties` as c
ON ST_CONTAINS(c.county_geom, ST_GEOGPOINT(a.longitude, a.latitude)) 
WHERE c.state_fips_code = '55'
GROUP BY c.county_name
""").to_dataframe()
county_applications_dict = q.set_index('county_name')['count'].to_dict()
county_applications_dict

{'Jefferson': 1,
 'Door': 7,
 'Dane': 18,
 'Columbia': 1,
 'Sheboygan': 2,
 'Kewaunee': 1,
 'Outagamie': 1,
 'Green Lake': 1,
 'Marinette': 2,
 'Brown': 2,
 'Barron': 2,
 'Bayfield': 2,
 'Walworth': 2,
 'Sauk': 1,
 'Oneida': 1,
 'Monroe': 1}

In [11]:
#q8
import time
bq.query("""
CREATE OR REPLACE MODEL `cs-544-398518.p8.loan_prediction_model`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['loan_amount']) 

AS

SELECT loan_amount, income, loan_term
FROM `cs-544-398518.p8.hdma`
""")
while True:
    if len(list(bq.list_models("cs-544-398518.p8"))) > 0:  # Hint: use bq.list_models()
        break
    time.sleep(5)
evaluate_model_sql = """
SELECT*
FROM ML.EVALUATE(MODEL `cs-544-398518.p8.loan_prediction_model`,
    (SELECT loan_amount, income, loan_term
    FROM cs-544-398518.p8.hdma
    ))
"""

result = bq.query(evaluate_model_sql).to_dataframe()["r2_score"][0]
result

0.29165346922392144

In [12]:
#q9
ds = bq.query("""
SELECT*
FROM
  ML.WEIGHTS(MODEL `cs-544-398518.p8.loan_prediction_model`)
""")
ds.to_dataframe()["weight"][0]

0.804631025398983

In [13]:
#q10
#q10
prediction_results = bq.query("""
WITH applications_with_id AS (
    SELECT *,ROW_NUMBER() OVER() AS row_id
    FROM `cs-544-398518.p8.applications`
)
SELECT a.loan_amount AS actual_loan_amount, predictions.predicted_loan_amount
FROM  ML.PREDICT(MODEL `cs-544-398518.p8.loan_prediction_model`, (SELECT income, 360 AS loan_term, row_id FROM applications_with_id)) AS predictions
JOIN applications_with_id AS a
ON predictions.row_id = a.row_id
""").to_dataframe()


greater_sum = (prediction_results['actual_loan_amount'] > prediction_results['predicted_loan_amount']).sum()
total_applications = len(prediction_results)

ratio = greater_sum / total_applications

ratio


0.6708860759493671